In [1]:
import tensorflow as tf
import pandas as pd

## The dataset

In [2]:
CSV_COLUMN_NAMES = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth', 'Species']
SPECIES = ['Setosa', 'Versicolor', 'Virginica']

In [3]:
train_path = tf.keras.utils.get_file(
    "iris_training.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv")
test_path = tf.keras.utils.get_file(
    "iris_test.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv")

train = pd.read_csv(train_path, names=CSV_COLUMN_NAMES, header=0)
test = pd.read_csv(test_path, names=CSV_COLUMN_NAMES, header=0)

8192/573 [============================================================================================================================================================================================================================================================================================================================================================================================================================================] - 0s 0us/step


In [5]:
train_y = train.pop('Species')
test_y = test.pop('Species')

In [71]:
_map = {
    0: 'Setosa',
    1: 'Versicolor',
    2: 'Virginica',
}
train_y = train_y.map(_map)
test_y = test_y.map(_map)

## Overview of programming with Estimators

To write a TensorFlow program based on pre-made Estimators, you must perform the following tasks:

- Create one or more input functions.
- Define the model's feature columns.
- Instantiate an Estimator, specifying the feature columns and various hyperparameters.
- Call one or more methods on the Estimator object, passing the appropriate input function as the source of the data.

## Create input functions

An __input function__ is a function that returns a `tf.data.Dataset` object which outputs the following two-element tuple:

- `features` - A Python dictionary in which:
    - Each key is the name of a feature.
    - Each value is an array containing all of that feature's values.
- `label` - An array containing the values of the label for every example.

In [74]:
def input_fn(features: pd.DataFrame, labels, training=True, batch_size=256):
    """An input function for training or evaluating"""
    dataset = tf.data.Dataset.from_tensor_slices((features.to_dict(orient='list'), labels))
    
    # Shuffle and repeat if you are in training mode.
    if training:
        dataset = dataset.shuffle(1000).repeat()
        
    return dataset.batch(batch_size)

## Define the feature columns

A feature column is an object describing how the model should use raw input data from the features dictionary. When you build an Estimator model, you pass it a list of feature columns that describes each of the features you want the model to use. The `tf.feature_column` module provides many options for representing data to the model.

In [75]:
feature_columns = []
for key in train.keys():
    feature_columns.append(tf.feature_column.numeric_column(key=key))

## Instantiate an estimator

Some premade classifier estimators:
- tf.estimator.DNNClassifier: for deep models that perform multi-class classification.
- tf.estimator.DNNLinearCombinedClassifier: for wide & deep models.
- tf.estimator.LinearClassifier: for classifiers based on linear models.

In [81]:
classifier = tf.estimator.DNNClassifier(
    hidden_units=[30, 10],
    feature_columns=feature_columns,
    n_classes=3,
    label_vocabulary=['Setosa', 'Versicolor', 'Virginica']
)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmptn4pk9dt', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


## Train, Evaluate, and Predict

In [82]:
classifier.train(
    input_fn=lambda: input_fn(train, train_y, training=True),
    hooks=None,
    steps=5000,
    max_steps=None,
    saving_listeners=None
)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving checkpoints for 0 into /tmp/tmptn4pk9dt/model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 0...
INFO:tensorflow:loss = 2.7911055, step = 0
INFO:tensorflow:global_step/sec: 339.461
INFO:tensorflow:loss = 1.4232947, step = 100 (0.297 sec)
INFO:tensorflow:global_step/sec: 416.526
INFO:tensorflow:loss = 1.1717352, step = 200 (0.239 sec)
INFO:tensorflow:global_step/sec: 414.599
INFO:tensorflow:loss = 1.056201, step = 300 (0.241 sec)
INFO:tensorflow:global_step/sec: 409.973
INFO:tensorflow:loss = 0.9975603, step = 400 (0.245 sec)
INFO:tensorflow:global_step/sec: 398.965
INFO:tensorflow:loss = 0.9520955, step = 500 (0.250 sec)
INFO:te

- `average_loss` (mean loss per sample)
- the `loss` (mean loss per mini-batch)
- estimator's `global_step` (the number of training iterations it underwent).

In [83]:
eval_results = classifier.evaluate(input_fn=lambda: input_fn(test, test_y, training=False))
eval_results

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2020-05-19T04:59:03Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmptn4pk9dt/model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 0.22765s
INFO:tensorflow:Finished evaluation at 2020-05-19-04:59:03
INFO:tensorflow:Saving dict for global step 5000: accuracy = 0.53333336, average_loss = 0.6261574, global_step = 5000, loss = 0.6261574
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 5000: /tmp/tmptn4pk9dt/model.ckpt-5000


{'accuracy': 0.53333336,
 'average_loss': 0.6261574,
 'loss': 0.6261574,
 'global_step': 5000}

In [103]:
# Generate predictions from the model
expected = ['Setosa', 'Versicolor', 'Virginica']
predict_x = {
    'SepalLength': [5.1, 5.9, 6.9],
    'SepalWidth': [3.3, 3.0, 3.1],
    'PetalLength': [1.7, 4.2, 5.4],
    'PetalWidth': [0.5, 1.5, 2.1],
}

def input_fn(features, batch_size=256):
    """An input function for prediction."""
    # Convert the inputs to a Dataset without labels.
    return tf.data.Dataset.from_tensor_slices(features).batch(batch_size)

predictions = classifier.predict(input_fn=lambda: input_fn(predict_x))

In [104]:
for pred_dict, expec in zip(predictions, expected):
    pred_class = pred_dict['classes'][0]
    probability = pred_dict['probabilities'][pred_dict['class_ids'][0]]
    
    print('Prediction is "{}" ({:.1f}%), expected "{}"'.format(pred_class.decode('UTF-8'), 100*probability, expec))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmptn4pk9dt/model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Prediction is "Setosa" (72.9%), expected "Setosa"
Prediction is "Virginica" (42.9%), expected "Versicolor"
Prediction is "Virginica" (69.9%), expected "Virginica"
